In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Tue Jan 31 07:43:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
os.chdir('/content/drive/MyDrive/Study/Data Science/AI 자연어 처리 전문가 양성 과정 6기/Project 3')

In [13]:
!pip -q install datasets sentence_transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.1 MB/s eta 0:00:00


In [17]:
from datasets import Dataset
from sentence_transformers import SentenceTransformer, util
from tqdm.auto import tqdm
from transformers import AutoTokenizer, pipeline

import torch
import numpy as np
import pandas as pd

In [54]:
kobart_s2w = 'kobart-base-v2-finetuned-spoken-to-written-v2_generated'
kobart_w2s = 'kobart-base-v2-finetuned-written-to-spoken-v2_generated'
t5_s2w = 't5-base-korean-paraphrase-finetuned-spoken-to-written_generated'
t5_w2s = 't5-base-korean-paraphrase-finetuned-written-to-spoken_generated'

In [53]:
def evaluate(dir, source_lang, target_lang):
    df = pd.read_csv(f'output/{dir}.csv').drop([0])
    print(df.shape)

    # 3. KLUE-NLI
    classifier = pipeline(
        "text-classification",
        model="Huffon/klue-roberta-base-nli",
        return_all_scores=True,
        device=0
    )
    tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
    sep_token = tokenizer.sep_token
    nli_label = []
    nli_score = []
    df['concat'] = df[target_lang].str.cat(df['generated'], sep=f' {sep_token} ')
    ds = Dataset.from_pandas(df)['concat']
    for output in tqdm(classifier(ds)):
        output_dict = {i['label']: i['score'] for i in output}
        label = max(output_dict.keys(), key=lambda x: output_dict[x])
        score = max(output_dict.values())
        nli_label.append(label)
        nli_score.append(score)
    df['nli_label'] = nli_label
    df['nli_score'] = nli_score
    df = df.drop(['concat'], axis=1)

    # 4. 코사인 유사도
    model = SentenceTransformer('ddobokki/klue-roberta-small-nli-sts')
    model.to(torch.device('cuda'))
    df['combine'] = df[target_lang].map(lambda x: [x]) + df['generated'].map(lambda x: [x])
    ds = Dataset.from_pandas(df)['combine']
    cosine_scores = []
    for sentences in tqdm(ds):
        embeddings = model.encode(sentences)
        #Compute cosine-similarities
        output = util.cos_sim(embeddings[0], embeddings[1]).cpu().item()
        cosine_scores.append(output)
    df['cosine_scores'] = cosine_scores
    df = df.drop(['combine'], axis=1)

    df = df.reset_index(drop=True)
    df.to_csv(f'output/{dir}-v2.csv', index=False)

    return df

In [55]:
kobart_s2w = evaluate(kobart_s2w, 'spoken', 'written')

(2754, 5)


/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


  0%|          | 0/2754 [00:00<?, ?it/s]

  0%|          | 0/2754 [00:00<?, ?it/s]

In [56]:
t5_s2w = evaluate(t5_s2w, 'spoken', 'written')

(2754, 5)


  0%|          | 0/2754 [00:00<?, ?it/s]

  0%|          | 0/2754 [00:00<?, ?it/s]

In [57]:
kobart_w2s = evaluate(kobart_w2s, 'written', 'spoken')

(2754, 5)


  0%|          | 0/2754 [00:00<?, ?it/s]

  0%|          | 0/2754 [00:00<?, ?it/s]

In [58]:
t5_w2s = evaluate(t5_w2s, 'written', 'spoken')

(2754, 5)


  0%|          | 0/2754 [00:00<?, ?it/s]

  0%|          | 0/2754 [00:00<?, ?it/s]

In [71]:
print("{:.3f}".format(kobart_s2w['nli_label'].value_counts()['ENTAILMENT']/2754))
print("{:.3f}".format(t5_s2w['nli_label'].value_counts()['ENTAILMENT']/2754))

0.920
0.931


In [72]:
print("{:.3f}".format(np.mean(kobart_s2w['cosine_scores'])))
print("{:.3f}".format(np.mean(t5_s2w['cosine_scores'])))

0.932
0.942


In [73]:
print("{:.3f}".format(kobart_w2s['nli_label'].value_counts()['ENTAILMENT']/2754))
print("{:.3f}".format(t5_w2s['nli_label'].value_counts()['ENTAILMENT']/2754))

0.943
0.967


In [74]:
print("{:.3f}".format(np.mean(kobart_w2s['cosine_scores'])))
print("{:.3f}".format(np.mean(t5_w2s['cosine_scores'])))

0.930
0.940
